# Toy modelling demonstration

This notebook demonstrates the use of several other scripts contained in this directory:
 - **[toy_data_generation.py](./toy_data_generation.py)**: generates "toy" datasets using known, specified generative processes
 - **[param_dict_generation.py](./param_dict_generation.py)**: generates parameter dicts specifying candidate models to train on the toy (or other, real) datasets
 - **[model_wrappers.py](./model_wrappers.py)**: presents a common interface for scikit-learn, statsmodels and XGBoost models, allowing multiple model types to be specified using the parameter dicts generated by param_dict_generation.py
 
The output is a csv file evaluating the performance of several different models on several toy datasets.

# Imports and settings

In [1]:
import multiprocessing
import pandas as pd
from sklearn.model_selection import train_test_split

from model_wrappers import try_models
from toy_data_generation import generate_linear_data
from param_dict_generation import generate_ols_list, generate_mlp_list, generate_xgb_list

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)

# Dataset generation

In [3]:
datasets = []
for nrows in [1000, 3000]:
    for nvars in [10, 30]:
        datasets.append((*generate_linear_data(nrows, nvars), f'linear_{nvars}vars_{nrows}rows'))
        print(f'Datasets with {nrows} rows and {nvars} vars successfully appended')

Datasets with 1000 rows and 10 vars successfully appended
Datasets with 1000 rows and 30 vars successfully appended
Datasets with 3000 rows and 10 vars successfully appended
Datasets with 3000 rows and 30 vars successfully appended


# Experiment list generation

## Statsmodels list

In [4]:
ols_list = generate_ols_list(alpha_list=[0.01, 1], L1_wt_list=[0, 0.5, 1])

statsmodels_param_list = ols_list

## Scikit list

In [5]:
n_cpus = multiprocessing.cpu_count()

In [6]:
mlp_list = generate_mlp_list(hidden_layout_list=[(200,), (50, 50)])

xgb_list = generate_xgb_list(depth_list = [3,5,7],
                             learning_rate_list = [0.03, 0.1, 0.3],
                             n_jobs=max(1, n_cpus-1),
                             objective='Linear',
                             early_stopping_rounds=10)

scikit_param_list = mlp_list

xgb_param_list = xgb_list

# Model training and evaluation

In [7]:
results = {}

for X, y, dataset_name in datasets:
    print(f'\n\n{dataset_name}\n')
    # update scikit_param_list for xgboost early stopping eval_sets if present
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=303)
    
    scikit_results = try_models(
            X_train, X_test, y_train, y_test,
            param_list=scikit_param_list, model='scikit'
        )
    for key in scikit_results:
        results[(dataset_name, key)] = scikit_results[key]
        
    xgb_results = try_models(
            X_train, X_test, y_train, y_test,
            param_list=xgb_param_list, model='xgb'
        )
    for key in xgb_results:
        results[(dataset_name, key)] = xgb_results[key]
        
    sm_results = try_models(
            X_train, X_test, y_train, y_test,
            param_list=statsmodels_param_list, model='statsmodels'
        )
    for key in sm_results:
        results[(dataset_name, key)] = sm_results[key]
    
display(pd.DataFrame(results).T)



linear_10vars_1000rows

MLP_200 trained in 1.13 seconds
MLP_50_50 trained in 0.63 seconds
XGB_Linear_maxd_3_lr_0.03 trained in 0.14 seconds
XGB_Linear_maxd_3_lr_0.1 trained in 0.02 seconds
XGB_Linear_maxd_3_lr_0.3 trained in 0.01 seconds
XGB_Linear_maxd_5_lr_0.03 trained in 0.04 seconds
XGB_Linear_maxd_5_lr_0.1 trained in 0.02 seconds
XGB_Linear_maxd_5_lr_0.3 trained in 0.01 seconds
XGB_Linear_maxd_7_lr_0.03 trained in 0.05 seconds
XGB_Linear_maxd_7_lr_0.1 trained in 0.02 seconds
XGB_Linear_maxd_7_lr_0.3 trained in 0.01 seconds
OLS_no_reg trained in 0.01 seconds
OLS_alpha_0.01_L1_0 trained in 0.01 seconds
OLS_alpha_0.01_L1_0.5 trained in 0.06 seconds
OLS_alpha_0.01_L1_1 trained in 0.05 seconds
OLS_alpha_1_L1_0 trained in 0.01 seconds
OLS_alpha_1_L1_0.5 trained in 0.02 seconds
OLS_alpha_1_L1_1 trained in 0.01 seconds


linear_30vars_1000rows

MLP_200 trained in 2.85 seconds
MLP_50_50 trained in 1.21 seconds
XGB_Linear_maxd_3_lr_0.03 trained in 0.26 seconds
XGB_Linear_maxd_3_lr_0.1 tra

mse_test  prediction_time  \
linear_10vars_1000rows MLP_200                    1.097753         0.001840   
                       MLP_50_50                  1.095565         0.001217   
                       XGB_Linear_maxd_3_lr_0.03  1.073700         0.001736   
                       XGB_Linear_maxd_3_lr_0.1   1.074997         0.001120   
                       XGB_Linear_maxd_3_lr_0.3   1.063121         0.001029   
                       XGB_Linear_maxd_5_lr_0.03  1.099480         0.001455   
                       XGB_Linear_maxd_5_lr_0.1   1.100323         0.001054   
                       XGB_Linear_maxd_5_lr_0.3   1.097820         0.001064   
                       XGB_Linear_maxd_7_lr_0.03  1.159591         0.001372   
                       XGB_Linear_maxd_7_lr_0.1   1.156559         0.001178   
                       XGB_Linear_maxd_7_lr_0.3   1.145096         0.001124   
                       OLS_no_reg                 1.058994         0.000870   
                       OLS_alpha_0.01_L1_0        1.055203         0.000631   
                       OLS_alpha_0.01_L1_0.5      1.058898         0.001233   
                       OLS_alpha_0.01_L1_1        1.062433         0.000875   
                       OLS_alpha_1_L1_0           1.556049         0.000903   
                       OLS_alpha_1_L1_0.5         2.050412         0.000825   
                       OLS_alpha_1_L1_1           2.338013         0.001300   
linear_30vars_1000rows MLP_200                    1.574922         0.001847   
                       MLP_50_50                  1.437539         0.001547   
                       XGB_Linear_maxd_3_lr_0.03  1.219188         0.003290   
                       XGB_Linear_maxd_3_lr_0.1   1.181296         0.001594   
                       XGB_Linear_maxd_3_lr_0.3   1.237171         0.001257   
                       XGB_Linear_maxd_5_lr_0.03  1.407986         0.003735   
                       XGB_Linear_maxd_5_lr_0.1   1.466349         0.001782   
                       XGB_Linear_maxd_5_lr_0.3   1.491472         0.001329   
                       XGB_Linear_maxd_7_lr_0.03  1.667895         0.004258   
                       XGB_Linear_maxd_7_lr_0.1   1.638856         0.001748   
                       XGB_Linear_maxd_7_lr_0.3   1.699303         0.001442   
                       OLS_no_reg                 0.955504         0.001376   
                       OLS_alpha_0.01_L1_0        0.984775         0.001305   
                       OLS_alpha_0.01_L1_0.5      0.988963         0.001328   
                       OLS_alpha_0.01_L1_1        0.983223         0.001292   
                       OLS_alpha_1_L1_0           3.841134         0.001072   
                       OLS_alpha_1_L1_0.5         5.012367         0.001167   
                       OLS_alpha_1_L1_1           6.025406         0.001170   
linear_10vars_3000rows MLP_200                    1.112194         0.004178   
                       MLP_50_50                  1.130811         0.002215   
                       XGB_Linear_maxd_3_lr_0.03  1.128375         0.004108   
                       XGB_Linear_maxd_3_lr_0.1   1.131541         0.001686   
                       XGB_Linear_maxd_3_lr_0.3   1.149915         0.001328   
                       XGB_Linear_maxd_5_lr_0.03  1.149652         0.004358   
                       XGB_Linear_maxd_5_lr_0.1   1.144911         0.001851   
                       XGB_Linear_maxd_5_lr_0.3   1.152557         0.001406   
                       XGB_Linear_maxd_7_lr_0.03  1.174615         0.003580   
                       XGB_Linear_maxd_7_lr_0.1   1.169416         0.001817   
                       XGB_Linear_maxd_7_lr_0.3   1.163329         0.001302   
                       OLS_no_reg                 1.093102         0.000634   
                       OLS_alpha_0.01_L1_0        1.111297         0.000614   
                       OLS_alpha_0.01_L1_0.5      1.108082         0.001611   
                     

# Output results

In [8]:
pd.DataFrame(results).T.to_csv('results.csv')